In [5]:
# test.py

import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# STSDataset 클래스는 앞서 수정한 내용을 사용합니다.
class STSDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, is_train=True):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_train = is_train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        sentence1 = str(self.df.loc[idx, 'sentence_1'])
        sentence2 = str(self.df.loc[idx, 'sentence_2'])
        
        inputs = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        
        input_ids = inputs['input_ids'].squeeze()  # [max_len]
        attention_mask = inputs['attention_mask'].squeeze()  # [max_len]
        
        item = {
            'input_ids': input_ids.to(device),
            'attention_mask': attention_mask.to(device),
        }
        
        if self.is_train:
            label = self.df.loc[idx, 'label']
            item['labels'] = torch.tensor(label, dtype=torch.float).to(device)
        
        return item
# 모델 클래스 정의 (이전과 동일)
class RegressionModel(nn.Module):
    def __init__(self, model_name):
        super(RegressionModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(self.config.hidden_size, self.config.hidden_size)
        self.activation = nn.ReLU()
        self.regressor = nn.Linear(self.config.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            pooled_output = outputs.pooler_output
        else:
            pooled_output = outputs.last_hidden_state[:, 0]
        
        x = self.dropout(pooled_output)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        outputs = self.regressor(x)
        return outputs.squeeze()


# 테스트 데이터 로드
test_df = pd.read_csv('/data/ephemeral/home/data/test.csv')  # 테스트 데이터 파일 경로를 입력하세요

# 토크나이저와 데이터셋 생성
max_len = 128
batch_size = 32

model_names = [
    'snunlp/KR-ELECTRA-discriminator',
    'beomi/KcELECTRA-base-v2022',
    'monologg/koelectra-base-v3-discriminator',
]

tokenizers = []
test_loaders = []

for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizers.append(tokenizer)
    
    test_dataset = STSDataset(test_df, tokenizer, max_len, is_train=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    test_loaders.append(test_loader)

# 테스트 데이터에 대한 예측 수행
test_predictions = []

for i, model_name in enumerate(model_names):
    print(f"\n=== 모델 {i+1}: {model_name} 테스트 데이터 예측 시작 ===")
    # 모델 생성 및 체크포인트 로드
    model = RegressionModel(model_name).to(device)

    # checkpoint = torch.load(f'/data/ephemeral/home/yujin/results/model_{i+1}.pt')
    # # 체크포인트에서 현재 모델과 크기가 일치하는 파라미터만 필터링
    # filtered_state_dict = {k: v for k, v in checkpoint.items() if k in model.state_dict() and model.state_dict()[k].shape == v.shape}
    # # 모델에 로드
    # model.load_state_dict(filtered_state_dict, strict=False)
    # try:
    #     # State_dict에서 불일치하는 파라미터를 건너뛰고 로드
    #     model.load_state_dict(filtered_state_dict, strict=False)
    # except RuntimeError as e:
    #     print(f"RuntimeError occurred: {e}")
    #     print("Continuing with the rest of the code...")
    
    model.load_state_dict(torch.load(f'/data/ephemeral/home/yujin/results/new_model_{i+1}.pt'), strict=False)
    model.eval()
    
    test_loader = test_loaders[i]
    
    preds = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Testing'):
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask']
            )
            preds.extend(outputs.cpu().numpy())
    
    test_predictions.append(preds)

# 앙상블 예측 (테스트 데이터)
ensemble_test_preds = np.mean(test_predictions, axis=0)
# 예측 값을 필요한 범위로 클리핑 (예: 0~5 사이)
ensemble_test_preds = np.clip(ensemble_test_preds, 0.0, 5.0)

ensemble_test_preds = np.round(ensemble_test_preds,1)

# 결과 저장
output_df = pd.DataFrame({
    'id': test_df['id'],
    'target': ensemble_test_preds
})

# CSV 파일로 저장
output_df.to_csv('/data/ephemeral/home/yujin/results/output_02.csv', index=False)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



=== 모델 1: snunlp/KR-ELECTRA-discriminator 테스트 데이터 예측 시작 ===


Testing: 100%|██████████| 35/35 [00:09<00:00,  3.73it/s]



=== 모델 2: beomi/KcELECTRA-base-v2022 테스트 데이터 예측 시작 ===


Testing: 100%|██████████| 35/35 [00:09<00:00,  3.75it/s]



=== 모델 3: monologg/koelectra-base-v3-discriminator 테스트 데이터 예측 시작 ===


Testing: 100%|██████████| 35/35 [00:02<00:00, 12.97it/s]
